**Integrantes del Grupo 19**



*   Juan David Palacios Feo
*   Gian Karlo Lanziano
*   Daniel Esteban Tobar




# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
import numpy as np

def jacobi_method(coef_matrix, target_vec, iter_count):
    size = len(target_vec)
    solution_vec = np.zeros(size)  # Initialize the solution vector

    for _ in range(iter_count):
        temp_vec = np.zeros(size)  # Initialize a new solution vector

        for j in range(size):
            sum_terms = np.dot(coef_matrix[j, :j], solution_vec[:j]) + np.dot(coef_matrix[j, j+1:], solution_vec[j+1:])
            temp_vec[j] = (target_vec[j] - sum_terms) / coef_matrix[j, j]

        solution_vec = temp_vec

    return solution_vec

def modified_jacobi_method(coef_matrix, target_vec, iter_count, stop_tolerance):
    size = len(target_vec)
    solution_vec = np.zeros(size)

    for iteration in range(iter_count):
        temp_vec = np.zeros(size)

        for j in range(size):
            sum_terms = np.dot(coef_matrix[j, :j], solution_vec[:j]) + np.dot(coef_matrix[j, j+1:], solution_vec[j+1:])
            temp_vec[j] = (target_vec[j] - sum_terms) / coef_matrix[j, j]

        norm_difference = np.linalg.norm(temp_vec - solution_vec)
        if norm_difference < stop_tolerance:
            print(f"Convergence reached after {iteration+1} iterations.")
            break

        solution_vec = temp_vec

    return solution_vec

result_jacobi = jacobi_method(A, b, n_iter)
result_modified_jacobi = modified_jacobi_method(A, b, n_iter, 0.1)


print("Result from jacobi_method:", result_jacobi)
print("Result from modified_jacobi_method:", result_modified_jacobi)


Convergence reached after 2 iterations.
Result from jacobi_method: [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
Result from modified_jacobi_method: [0.05049637 0.04568193 0.05285884 0.03683105 0.06193394 0.0092481
 0.00191869 0.03822729 0.00729613 0.06067932]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:


import numpy as np

def gauss_seidel_method(coef_matrix, solution_vector, iterations):
    length = len(coef_matrix)
    current_solution = np.zeros(length)  # Start with an initial guess of zeros

    for _ in range(iterations):
        for index in range(length):
            sum1 = np.dot(coef_matrix[index, :index], current_solution[:index])
            sum2 = np.dot(coef_matrix[index, index + 1:], current_solution[index + 1:])
            current_solution[index] = (solution_vector[index] - sum1 - sum2) / coef_matrix[index, index]

    matrix_residual = coef_matrix @ current_solution - solution_vector
    norm_iter_matrix = np.linalg.norm(coef_matrix - np.diag(np.diag(coef_matrix)))

    return current_solution, matrix_residual, norm_iter_matrix

dimension = 10
random_matrix = np.random.rand(dimension, dimension)
random_vector = np.random.rand(dimension)

iteration = 100

gauss_seidel_solution = gauss_seidel_method(random_matrix, random_vector, iteration)

print("Gauss-Seidel Solution:", gauss_seidel_solution)


Gauss-Seidel Solution: (array([-6.40764666e+224,  1.46016855e+226, -3.62304513e+227,
        3.58025637e+227, -3.35520016e+226,  7.09046811e+227,
       -7.18761268e+227, -1.29999691e+228,  5.83787225e+227,
        2.46893027e+227]), array([-8.96339445e+226,  5.88189786e+227, -6.41756694e+226,
        1.37040517e+227, -9.17541871e+226, -1.17174729e+228,
       -6.16236017e+227,  7.19993140e+227,  2.23897797e+227,
       -2.93072054e-001]), 5.90790386858141)


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
import numpy as np

def min_resid_method(coeff_matrix, target_vector, iterations):
    dimension = len(target_vector)
    sol_vector = np.zeros(dimension)

    for _ in range(iterations):
        resid_vector = target_vector - coeff_matrix.dot(sol_vector)
        iter_param = np.dot(resid_vector, resid_vector) / np.dot(resid_vector, coeff_matrix.dot(resid_vector))
        sol_vector += iter_param * resid_vector

    return sol_vector


dimension = 10
random_coeff_matrix = np.random.rand(dimension, dimension)

# Generate a random vector
random_target_vector = np.random.rand(dimension)

# Set the number of iterations
iteration_count = 100

# Perform the minimum residual method
min_resid_solution = min_resid_method(random_coeff_matrix, random_target_vector, iteration_count)


true_solution = np.linalg.solve(random_coeff_matrix, random_target_vector)


solution_residual = random_target_vector - random_coeff_matrix.dot(min_resid_solution)


residual_norm = np.linalg.norm(solution_residual)

solution_deviation = np.linalg.norm(min_resid_solution - true_solution)

print("Minimum Residual Solution:", min_resid_solution)
print("Norm of the residual:", residual_norm)
print("Deviation from the ground truth solution:", solution_deviation)



Minimum Residual Solution: [-8.94828789e+11 -1.77487462e+13 -5.14776162e+12 -4.82211424e+12
  6.85370409e+12  2.05703206e+13  1.25193914e+13 -6.26045224e+12
 -1.63959166e+12 -1.87255501e+13]
Norm of the residual: 48174013499219.13
Deviation from the ground truth solution: 37214851995890.45
